# 変換のテスト

In [1]:
import sys
sys.path.append("..")

import torch
import numpy as np
import networkx as nx
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

from glob import glob
from tqdm import tqdm
from collections import Counter
from lmfit.models import PowerLawModel
from scipy.stats import skew, kurtosis

from reserch_utils_HT import no_growth, no_preferential_attachment
from reserch_utils_HT import network_to_image, degree_hist
from model import Generator
from model import Discriminator
from train_utils import get_loader, label2onehot

In [8]:
def img_transform(model, img, transform, device):
    """ make all label fake image from graph """
    graph_tensor = transform(img).reshape(1,1,100,100).to(device) # img to tensor
    trg_label = label2onehot(torch.tensor([0,1,2,3]), 4).to(device) # make target label
    origin_tensors = torch.cat([graph_tensor for _ in range(4)]) # cat origin image tensor
    # generate fake image
    with torch.no_grad():
        fake_tensor = model(origin_tensors, trg_label)
    return fake_tensor.detach().cpu()

def power_law_fit(G):
    """ degree dist fitting """
    c = Counter(dict(G.degree()).values())
    score_sorted = sorted(c.items(), key=lambda x:x[0])
    x = [k for k, v in score_sorted if k!=0]
    y = [v for k, v in score_sorted if k!=0]
    model = PowerLawModel()
    params = model.guess(y, x=x)
    result = model.fit(y, params, x=x)
    return result.values

def tensor_to_graph(tensor, thres):
    A = (tensor + tensor.T).numpy()
    A = (A > thres).astype(int)
    G = nx.from_numpy_array(A)
    return G

def net_info(G):
    def __average_path_len(G):
        if nx.is_connected(G):
            return nx.average_shortest_path_length(G)
        else:
            return None
    
    degs = np.array(list(dict(G.degree()).values()))
    info = {
        "max_degree" : max(degs),
        "min_degree" : min(degs),
        "average_degree": np.average(degs),
        "num_edges" : nx.number_of_edges(G),
        "beta" : power_law_fit(G)['exponent'], # べきfittingの指数
        "variance" : np.var(degs,ddof=1),
        "mode" : Counter(degs).most_common()[0][0],
        "Skewness": skew(degs),  # 歪度
        "Kurtosis": kurtosis(degs),  # 尖度
        "average_cluster": nx.average_clustering(G),  # 平均クラスター
        "average_shortest_path": __average_path_len(G)  # 平均最短経路長
    }
    return info


def test_3model(model,transform):
    df = pd.DataFrame()
    num_to_kind = ["BA", "Attach", "Growth", "Random"]
    for m in tqdm([1,2,3,4,5,6,7,8,9,10]):
        for i in range(100):
            G = nx.barabasi_albert_graph(100, m, seed=i+2000)
            info = net_info(G)
            info["origin_label"] = "BA"
            info["fake_label"] = "origin"
            info["parameter"] = m
            info["seed"] = i+2000
            df = df.append(info, ignore_index=True)

            img = network_to_image(G, sort=True)
            fake_tensor = img_transform(model, img, transform, device)

            for i in range(4):
                fake_G = tensor_to_graph(fake_tensor[i][0], 1.)
                info = net_info(fake_G)
                info["origin_label"] = "BA"
                info["fake_label"] = num_to_kind[i]
                info["parameter"] = m
                info["seed"] = i+2000
                df = df.append(info, ignore_index=True)

    for m in tqdm([1,2,3,4,5,6,7,8,9,10]):
        for i in range(100):
            G = no_preferential_attachment(100, m, seed=i+2000)
            info = net_info(G)
            info["origin_label"] = "Growth"
            info["fake_label"] = "origin"
            info["figsize=(24, 12)parameter"] = m
            info["seed"] = i+2000
            df = df.append(info, ignore_index=True)

            img = network_to_image(G, sort=True)
            fake_tensor = img_transform(model, img, transform, device)

            for i in range(4):
                fake_G = tensor_to_graph(fake_tensor[i][0], 1.)
                info = net_info(fake_G)
                info["origin_label"] = "Growth"
                info["fake_label"] = num_to_kind[i]
                info["parameter"] = m
                info["seed"] = i+2000
                df = df.append(info, ignore_index=True)


    for m in tqdm([100, 200, 300,400,500,600,700,800,900,1000]):
        for i in range(100):
            G = no_growth(100, m, seed=i+2000)
            info = net_info(G)
            info["origin_label"] = "Attach"
            info["fake_label"] = "origin"
            info["parameter"] = m
            info["seed"] = i+2000
            df = df.append(info, ignore_index=True)

            img = network_to_image(G, sort=True)
            fake_tensor = img_transform(model, img, transform, device)

            for i in range(4):
                fake_G = tensor_to_graph(fake_tensor[i][0], 1.)
                info = net_info(fake_G)
                info["origin_label"] = "Attach"
                info["fake_label"] = num_to_kind[i]
                info["parameter"] = m
                info["seed"] = i+2000
                df = df.append(info, ignore_index=True)
    return df

In [9]:
transform = transforms.Compose(
    [
        transforms.Resize(100),
        transforms.Grayscale(),
        transforms.ToTensor(),
    ]
)

In [16]:
sorted(glob("../logs/*"))[4:]

['../logs/subset1',
 '../logs/subset1_edge0.5',
 '../logs/subset1_edge1',
 '../logs/subset1_edge10']

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# config
image_size = 100
c_dim = 4
g_repeat_num = 6
g_conv_dim = 64

for file_name in sorted(glob("../logs/*"))[4:]:
    for epoch in [50000, 90000, 100000]:
        print(file_name, epoch)
        # set generator
        model = Generator(g_conv_dim, c_dim, g_repeat_num).to(device)
        model_path = f"{file_name}/model/{epoch}-G.ckpt"
        model.load_state_dict(torch.load(model_path))

        df = test_3model(model, transform)
        df.to_csv(f"{file_name}/test_{epoch}.csv", index=False)


  0%|          | 0/10 [00:00<?, ?it/s]

../logs/subset1 50000



  0%|          | 0/10 [00:00<?, ?it/s]

../logs/subset1 90000



  0%|          | 0/10 [00:00<?, ?it/s]

...


  0%|          | 0/10 [00:00<?, ?it/s]

../logs/subset1_edge10 90000



  0%|          | 0/10 [00:00<?, ?it/s]

../logs/subset1_edge10 100000



100%|██████████| 10/10 [02:18<00:00, 17.10s/it]

In [12]:
glob("../logs/*")

['../logs/subset1_edge10',
 '../logs/poisson',
 '../logs/poisson_edge10',
 '../logs/poisson_edge1',
 '../logs/subset1',
 '../logs/poisson_edge0.5',
 '../logs/subset1_edge1']

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# config
image_size = 100
c_dim = 4
g_repeat_num = 6
g_conv_dim = 64

file_name = '../logs/poisson'
for epoch in [100000]:
    print(file_name, epoch)
    # set generator
    model = Generator(g_conv_dim, c_dim, g_repeat_num).to(device)
    model_path = f"{file_name}/model/{epoch}-G.ckpt"
    model.load_state_dict(torch.load(model_path))

    df = test_3model(model, transform)
    df.to_csv(f"{file_name}/test_{epoch}.csv", index=False)

../logs/poisson 100000



100%|██████████| 10/10 [04:29<00:00, 35.39s/it]